In [1]:
#fundamental tools for the game environment model
import numpy as np
from multiprocess import Process, Queue, Semaphore, Lock
import time

# for visualations
import matplotlib.pyplot as plt

In [2]:
from randomchooser import Chooser
from exploresims import simulate_explore

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
max_processes = 6
sem = Semaphore(max_processes)
out_lock = Lock()
output_queue = Queue()
pos_queue = Queue()

In [4]:
# make positions available
num_cols = 2
for i in range(max_processes):
    x = 30 + (i%num_cols) * (610 + 30) 
    y = 40 + (i//num_cols)*(300 + 40)
    pos_queue.put((x,y))

In [5]:
choosers = []
choosers.append(Chooser(5))
for i in range(11):
    choosers.append(Chooser(5, [1,10,10,10,1], 1+i))

In [6]:
sims = []
for ch in choosers:
    curr_args = [sem, out_lock, output_queue, 100,100,40, ch, 21,
                 300, 15, 1.25, True, pos_queue]
    p = Process(target=simulate_explore, args=curr_args)
    sims.append(p)

In [7]:
for sim in sims:
    sim.start()

In [8]:
results = []
while not output_queue.empty():
    results.append(output_queue.get())

In [9]:
len(results)

12

In [13]:
weights = []
durations = []
win_data = []
stdev_data = []
trace_averages = []
for result in results:
    weights.append(result[0])
    durations.append(result[1])
    win_data.append(result[2])
    stdev_data.append(result[3])
    trace_averages.append(result[4])

In [15]:
win_averages = []
win_stds = []
win_errors = []
for win in win_data:
    win_averages.append(win[0])
    win_stds.append(win[1])
    win_errors.append(win[2])

In [17]:
stdev_averages = []
stdev_stds = []
stdev_errors = []
for stdev in stdev_data:
    stdev_averages.append(stdev[0])
    stdev_stds.append(stdev[1])
    stdev_errors.append(stdev[2])

In [20]:
for (mean, error) in zip(stdev_averages, stdev_errors):
    print(mean, error)

21.86602532982324 0.10469247628252913
15.313870940615198 0.07724158159185099
15.716533999515704 0.09198569511556216
18.010179063609005 0.08559797513084456
19.47236847485734 0.09431429254552265
21.842453700688335 0.09460410394343419
16.555809859094502 0.08570969776239723
13.63678957677643 0.06964675739907034
13.893293978274947 0.07674123096475867
13.548180713872936 0.08147844570211402
14.818840616326467 0.07659644601206021
14.055505151820253 0.0800953992967946


In [3]:
ch1 = Chooser(5)
ch2 = Chooser(5, [1,10,10,10,1])

In [5]:
lol1 = Process(target=simulate_explore, args=[sem, out_lock, output_queue, 100,100,40, ch1, 15, 1.08])
lol2 = Process(target=simulate_explore, args=[sem, out_lock, output_queue, 100,100,40, ch2, 15, 1.08])

In [6]:
lol1.start()
lol2.start()

In [8]:
lol1.join()
lol2.join()